<a href="https://colab.research.google.com/github/keysenelvial/dinam/blob/main/dinam2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №2**

Провести классификацию(не регрессию) найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

# Загрузка данных
data = pd.read_csv('/content/drive/MyDrive/train 3.csv', on_bad_lines='skip')
print(data.head())


         id  log_price property_type        room_type  \
0   6901257   5.010635     Apartment  Entire home/apt   
1   6304928   5.129899     Apartment  Entire home/apt   
2   7919400   4.976734     Apartment  Entire home/apt   
3  13418779   6.620073         House  Entire home/apt   
4   3808709   4.744932     Apartment  Entire home/apt   

                                           amenities  accommodates  bathrooms  \
0  {"Wireless Internet","Air conditioning",Kitche...             3        1.0   
1  {"Wireless Internet","Air conditioning",Kitche...             7        1.0   
2  {TV,"Cable TV","Wireless Internet","Air condit...             5        1.0   
3  {TV,"Cable TV",Internet,"Wireless Internet",Ki...             4        1.0   
4  {TV,Internet,"Wireless Internet","Air conditio...             2        1.0   

   bed_type cancellation_policy  cleaning_fee  ...   latitude   longitude  \
0  Real Bed              strict          True  ...  40.696524  -73.991617   
1  Real Bed     

In [12]:
# Предобработка данных
data = data[['log_price', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'property_type']].dropna()

# Уменьшение размера датасета: выбор 15% случайных строк
sampled_data = data.sample(frac=0.15, random_state=42)  # используйте random_state для воспроизводимости

# Разделение цены на категории
sampled_data['price_category'] = pd.qcut(sampled_data['log_price'], q=4, labels=False)

# Подготовка признаков и целевой переменной
X = sampled_data[['accommodates', 'bathrooms', 'bedrooms', 'beds']]
y = sampled_data['price_category']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Вывод границ категорий цен
print("Границы категорий цен:", pd.qcut(data['log_price'], q=4).unique())


Границы категорий цен: [(4.7, 5.165], (4.304, 4.7], (-0.001, 4.304], (5.165, 7.6]]
Categories (4, interval[float64, right]): [(-0.001, 4.304] < (4.304, 4.7] < (4.7, 5.165] <
                                           (5.165, 7.6]]


In [15]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

params = {
    'svc__C': [0.1, 1, 10, 100],  # Диапазон для параметра регуляризации C
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid']  # Разные виды ядер
}

grid_search = GridSearchCV(pipe, param_grid=params, cv=5,  n_jobs=-1)


In [16]:
# Обучение модели с использованием GridSearch
grid_search.fit(X_train, y_train)

print('Лучшие параметры:', grid_search.best_params_)

Лучшие параметры: {'svc__C': 1, 'svc__kernel': 'rbf'}


In [17]:
# Обучение модели с лучшими параметрами
best_svc = grid_search.best_estimator_
best_svc.fit(X_train, y_train)


y_pred = best_svc.predict(X_test)


print(classification_report(y_test, y_pred))


print("Границы категорий цен:", pd.qcut(data['log_price'], q=4).unique())

              precision    recall  f1-score   support

           0       0.39      0.80      0.52       404
           1       0.32      0.14      0.20       459
           2       0.39      0.27      0.32       423
           3       0.67      0.61      0.63       428

    accuracy                           0.45      1714
   macro avg       0.44      0.46      0.42      1714
weighted avg       0.44      0.45      0.41      1714

Границы категорий цен: [(4.7, 5.165], (4.304, 4.7], (-0.001, 4.304], (5.165, 7.6]]
Categories (4, interval[float64, right]): [(-0.001, 4.304] < (4.304, 4.7] < (4.7, 5.165] <
                                           (5.165, 7.6]]
